In [ ]:
!pip install jupyterlab-widgets geopandas 

In [19]:
# @title Select the whl file for the wawbus package
from ipywidgets import FileUpload
from IPython.display import display
upload = FileUpload(accept='.whl', multiple=False)
display(upload)

FileUpload(value=(), accept='.whl', description='Upload')

In [ ]:
!pip install {list(upload.value.keys())[0]}

In [12]:
from wawbus import WawBus
import numpy as np
import pandas as pd
import geopandas as gpd

from ipywidgets import Dropdown, Text, VBox, IntSlider

In [13]:
# @title Main api configuration
dataset = Dropdown(options=['20240207-66k', 'None'], description='Prefetched dataset')
api_key = Text(description='API key (optional)', placeholder='Enter your API key')
collection = IntSlider(description='How many data points to collect. If zero, only the prefetched dataset will be used', min=0, step=1)
display(VBox([dataset, api_key, collection]))

In [14]:
api = WawBus(apikey=api_key.value, dataset=dataset.value if dataset.value != 'None' else None)

In [18]:
if collection.value > 0:
    api.collect_positions(collection.value)

ValueError: API key is required.

In [16]:
api.dataset.head()

,Lines,Lon,VehicleNumber,Time,Lat,Brigade
0,213,21.215783,1000,2024-02-07 18:08:33,52.160193,1
1,219,21.151121,1001,2024-02-07 18:08:47,52.186415,2
2,225,21.115344,1002,2024-02-07 17:39:55,52.234625,503
3,219,21.166092,1003,2024-02-07 18:08:46,52.204143,1
4,213,21.102482,1004,2024-02-07 18:08:52,52.222826,3


In [17]:
df = api.calculate_speed()
df.head()

,Lines,Lon,VehicleNumber,Time,Lat,Brigade,Speed
0,213,21.215783,1000,2024-02-07 18:08:33,52.160193,1,28.259437
1,219,21.151121,1001,2024-02-07 18:08:47,52.186415,2,42.389216
2,225,21.115344,1002,2024-02-07 17:39:55,52.234625,503,NaN
3,219,21.166092,1003,2024-02-07 18:08:46,52.204143,1,7.906190
4,213,21.102482,1004,2024-02-07 18:08:52,52.222826,3,0.354967
